# **Homework 2: Phoneme Classification**


Objectives:
* Solve a classification problem with deep neural networks (DNNs).
* Understand recursive neural networks (RNNs).

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

# Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`: training metadata
- `libriphone/train_labels`: training labels
- `libriphone/test_split.txt`: testing metadata
- `libriphone/feat/train/*.pt`: training feature
- `libriphone/feat/test/*.pt`:  testing feature

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2023spring-hw2/data) and upload it to the workspace.**


In [1]:
# #!pip install --upgrade gdown

# # Main link
# !gdown --id '1N1eVIDe9hKM5uiNRGmifBlwSDGiVXPJe' --output libriphone.zip
# # !gdown --id '1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1' --output libriphone.zip

# !unzip -q libriphone.zip
# !ls libriphone

# Some Utility Functions
**Fixes random number generator seeds for reproducibility.**

In [2]:
import numpy as np
import torch
import random

def same_seeds(seed):
    random.seed(seed) 
    np.random.seed(seed)  
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [3]:
# import os
# import torch
# from tqdm import tqdm

# def load_feat(path):
#     feat = torch.load(path)
#     return feat

# def shift(x, n):
#     if n < 0:
#         #md，看见下面n是负的脑子没转过弯，是因为判断了n小于0，所以取负号变成正的
#         left = x[0].repeat(-n, 1)
#         right = x[:n]
#     elif n > 0:
#         right = x[-1].repeat(n, 1)
#         left = x[n:]
#     else:
#         return x

#     return torch.cat((left, right), dim=0)

# def concat_feat(x, concat_n):
#     assert concat_n % 2 == 1 # n must be odd
#     if concat_n < 2:
#         return x
#     seq_len, feature_dim = x.size(0), x.size(1)
#     x = x.repeat(1, concat_n) 
#     x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
#     mid = (concat_n // 2)
#     for r_idx in range(1, mid+1):
#         x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
#         x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

#     return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

# def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8):
#     class_num = 41 # NOTE: pre-computed, should not need change

#     if split == 'train' or split == 'val':
#         mode = 'train'
#     elif split == 'test':
#         mode = 'test'
#     else:
#         raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

#     label_dict = {}
#     if mode == 'train':
#         for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
#             line = line.strip('\n').split(' ')
#             label_dict[line[0]] = [int(p) for p in line[1:]]
        
#         # split training and validation data
#         usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
#         random.shuffle(usage_list)
#         train_len = int(len(usage_list) * train_ratio)
#         usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

#     elif mode == 'test':
#         usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

#     usage_list = [line.strip('\n') for line in usage_list]
#     print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

#     max_len = 3000000
#     X = torch.empty(max_len, 39 * concat_nframes)
#     if mode == 'train':
#         y = torch.empty(max_len, dtype=torch.long)

#     idx = 0
#     for i, fname in tqdm(enumerate(usage_list)):
#         feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
#         cur_len = len(feat)
#         feat = concat_feat(feat, concat_nframes)
#         if mode == 'train':
#           label = torch.LongTensor(label_dict[fname])

#         X[idx: idx + cur_len, :] = feat
#         if mode == 'train':
#           y[idx: idx + cur_len] = label

#         idx += cur_len

#     X = X[:idx, :]
#     if mode == 'train':
#       y = y[:idx]

#     print(f'[INFO] {split} set')
#     print(X.shape)
#     if mode == 'train':
#       print(y.shape)
#       return X, y
#     else:
#       return X

import os
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

# def concat_feat(x, concat_n):
#     assert concat_n % 2 == 1 # n must be odd
#     if concat_n < 2:
#         return x
#     seq_len, feature_dim = x.size(0), x.size(1)
#     x = x.repeat(1, concat_n) 
#     x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
#     mid = (concat_n // 2)
#     for r_idx in range(1, mid+1):
#         x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
#         x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

#     return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)
#deepseek优化向量拼接
def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2: 
        return x
    
    seq_len, feat_dim = x.shape
    mid = concat_n // 2
    device = x.device
    
    # 创建位移索引模板
    indices = torch.arange(seq_len, device=device)[:, None] + torch.arange(-mid, mid+1, device=device)
    indices = torch.clamp(indices, 0, seq_len-1)  # 边界限制
    
    # 通过索引直接获取拼接特征
    stacked = x[indices.view(-1)].view(seq_len, concat_n, feat_dim)
    return stacked.view(seq_len, -1)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8):
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]
        
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        train_list = usage_list[:train_len]
        val_list = usage_list[train_len:]

    elif mode == 'test':
        test_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

    # 略微魔改了一下关于数据划分的代码，避免出现训练验证集交叉的可能
    def transform_data(usage_list, split):
        usage_list = [line.strip('\n') for line in usage_list]
        print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

        max_len = 3000000
        X = torch.empty(max_len, 39 * concat_nframes)
        if mode == 'train':
            y = torch.empty(max_len, dtype=torch.long)

        idx = 0
        for i, fname in tqdm(enumerate(usage_list)):
            feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
            cur_len = len(feat)
            feat = concat_feat(feat, concat_nframes)
            if mode == 'train':
                label = torch.LongTensor(label_dict[fname])

            X[idx: idx + cur_len, :] = feat
            if mode == 'train':
                y[idx: idx + cur_len] = label

            idx += cur_len

        X = X[:idx, :]
        if mode == 'train':
            y = y[:idx]

        print(f'[INFO] {split} set')
        print(X.shape)
        if mode == 'train':
            print(y.shape)
            return X, y
        else:
            return X
    
    if mode == 'test':
        return transform_data(test_list, "test")
    
    return *transform_data(train_list, "train"), *transform_data(val_list, "val")

# Dataset

In [4]:
import torch
from torch.utils.data import Dataset

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


# Model
Feel free to modify the structure of the model.

In [5]:
# import torch.nn as nn

# class BasicBlock(nn.Module):
#     def __init__(self, input_dim, output_dim):
#         super(BasicBlock, self).__init__()

#         # TODO: apply batch normalization and dropout for strong baseline.
#         # Reference: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html (batch normalization)
#         #       https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html (dropout)
#         self.block = nn.Sequential(
#             nn.Linear(input_dim, output_dim),
#             nn.BatchNorm1d(output_dim),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#         )

#     def forward(self, x):
#         x = self.block(x)
#         return x


# class Classifier(nn.Module):
#     def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
#         super(Classifier, self).__init__()

#         self.fc = nn.Sequential(
#             BasicBlock(input_dim, hidden_dim),
#             *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
#             nn.Linear(hidden_dim, output_dim)
#         )
        
#         self.emb = nn.Embedding(input_dim, embedding_dim=hidden_dim)
#         self.rnn = nn.RNN(hidden_dim, hidden_dim, num_layers=1)
#         self.linear = nn.Linear(hidden_dim, output_dim)

#         # self.rnn = nn.Sequential(
#         #     nn.Embedding(input_dim, embedding_dim=hidden_dim),
#         #     nn.RNN(hidden_dim, hidden_dim, num_layers=1),
#         #     nn.Linear(hidden_dim, output_dim),
#         # )

#     def forward(self, x):
#         # x = self.fc(x)
#         x = self.rnn(x)
#         print(x.shape())
#         return x

# Hyper-parameters

In [6]:
# data prarameters
# TODO: change the value of "concat_nframes" for medium baseline
concat_nframes = 85   # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.95   # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 1213          # random seed
batch_size = 512        # batch size
num_epoch = 30         # the number of training epoch
learning_rate = 1e-4      # learning rate
model_path = './model.ckpt'  # the path where the checkpoint will be saved

# model parameters
# TODO: change the value of "hidden_layers" or "hidden_dim" for medium baseline
input_dim = 39 * concat_nframes  # the input dim of the model, you should not change the value
hidden_layers = 6          # the number of hidden layers
hidden_dim = 600           # the hidden dim

# Dataloader

In [7]:
from torch.utils.data import DataLoader
import gc

same_seeds(seed)
torch.cuda.set_device("cuda:7")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

# # preprocess data
# train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
# val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
train_X, train_y, val_X, val_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)


# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

DEVICE: cuda
[Dataset] - # phone classes: 41, number of utterances for train: 3257


0it [00:00, ?it/s]/tmp/ipykernel_544156/449328754.py:100: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  feat = torch.load(path)
3257it [00:37, 87.60it/s] 


[INFO] train set
torch.Size([2007632, 3315])
torch.Size([2007632])
[Dataset] - # phone classes: 41, number of utterances for val: 172


172it [00:02, 60.86it/s] 

[INFO] val set
torch.Size([109162, 3315])
torch.Size([109162])


# Training

In [8]:
# # data prarameters
# # TODO: change the value of "concat_nframes" for medium baseline
# concat_nframes = 25   # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
# train_ratio = 0.75   # the ratio of data used for training, the rest will be used for validation

# # training parameters
# seed = 1213          # random seed
# batch_size = 512        # batch size
# num_epoch = 30         # the number of training epoch
# learning_rate = 1e-4      # learning rate
# model_path = './model.ckpt'  # the path where the checkpoint will be saved

# # model parameters
# # TODO: change the value of "hidden_layers" or "hidden_dim" for medium baseline
# input_dim = 39 * concat_nframes  # the input dim of the model, you should not change the value
# hidden_layers = 3          # the number of hidden layers
# hidden_dim = 1024           # the hidden dim

In [ ]:
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        # TODO: apply batch normalization and dropout for strong baseline.
        # Reference: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html (batch normalization)
        #       https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html (dropout)
        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
        )

    def forward(self, x):
        x = self.block(x)
        return x


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()

        # # fc fully connected
        # self.fc = nn.Sequential(
        #     BasicBlock(input_dim, hidden_dim),
        #     # * 解包操作符，例如*args，等同于手动输入列表中的元素，去掉列表的框
        #     *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
        #     nn.Linear(hidden_dim, output_dim)
        # )
        self.rnn = nn.LSTM(39, hidden_dim, num_layers=hidden_layers, batch_first=True, bidirectional=True, dropout=0.5)
        # 用bidiretional维度会翻倍，这是初版输出层，用了biLSTM输出的所有信息
        # self.linear = nn.Linear(input_dim//39 * hidden_dim * 2, output_dim)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim//2),
            nn.BatchNorm1d(hidden_dim//2),
            nn.ReLU(),
            #作为LSTM的全连接层时，层数调小，dropout也调小
            nn.Dropout(0.1),
            nn.Linear(hidden_dim//2, output_dim),
        )
        # self.rnn = nn.Sequential(
        #     #nn.Embedding(input_dim, embedding_dim=hidden_dim),
        #     nn.RNN(input_dim, hidden_dim, num_layers=1),
        #     nn.Linear(hidden_dim, output_dim),
        # )

    def forward(self, x):
        # x = self.fc(x)
        batch_size, x_input = x.shape
        x, _ = self.rnn(x.view(batch_size,-1,39))
        # x = self.fc(x.reshape(batch_size, -1))
        #只取x的最后一个时间步作为fc的输入,注意正向和反向数据的位置
        x = x[:,input_dim//78]
        # print(x.shape)
        x = self.fc(x)
        return x

In [10]:
# create model, define a loss function, and optimizer
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
#已经训练30轮
# model.load_state_dict(torch.load(model_path))
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad() 
        outputs = model(features) 
        
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    
    # validation
    model.eval() # set the model to evaluation mode
    with torch.no_grad():
        for i, batch in enumerate(tqdm(val_loader)):
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)
            outputs = model(features)
            
            loss = criterion(outputs, labels) 
            
            _, val_pred = torch.max(outputs, 1) 
            val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
            val_loss += loss.item()

    print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Acc: {train_acc/len(train_set):3.5f} Loss: {train_loss/len(train_loader):3.5f} | Val Acc: {val_acc/len(val_set):3.5f} loss: {val_loss/len(val_loader):3.5f}')

    # if the model improves, save a checkpoint at this epoch
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f'saving model with acc {best_acc/len(val_set):.5f}')


100%|██████████| 214/214 [00:16<00:00, 13.07it/s]


[001/030] Train Acc: 0.75314 Loss: 0.81794 | Val Acc: 0.80078 loss: 0.65145
saving model with acc 0.80078


100%|██████████| 214/214 [00:16<00:00, 13.09it/s]


[002/030] Train Acc: 0.87874 Loss: 0.36299 | Val Acc: 0.81543 loss: 0.66507
saving model with acc 0.81543


100%|██████████| 214/214 [00:16<00:00, 13.04it/s]


...

[028/030] Train Acc: 0.98415 Loss: 0.04053 | Val Acc: 0.84180 loss: 1.11270


100%|██████████| 214/214 [00:16<00:00, 13.13it/s]


[029/030] Train Acc: 0.98481 Loss: 0.03930 | Val Acc: 0.83936 loss: 1.12358


100%|██████████| 214/214 [00:16<00:00, 12.97it/s]

[030/030] Train Acc: 0.98518 Loss: 0.03822 | Val Acc: 0.84021 loss: 1.14847


In [11]:
# del train_set, val_set
# del train_loader, val_loader
# gc.collect()

# Testing
Create a testing dataset, and load model from the saved checkpoint.

In [12]:
# load data
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 857


0it [00:00, ?it/s]/tmp/ipykernel_544156/449328754.py:100: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  feat = torch.load(path)
857it [00:24, 35.47it/s] 

[INFO] test set
torch.Size([527364, 3315])


In [13]:
# load model
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
model.load_state_dict(torch.load(model_path))

/tmp/ipykernel_544156/3352857437.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

Make prediction.

In [14]:
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 1031/1031 [01:16<00:00, 13.48it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [15]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))